In [1]:
import tensorflow as tf

In [2]:
import math

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
x=mnist.train.images
y=mnist.train.labels

In [5]:
mnist.test.images.shape

(10000, 784)

In [6]:
print (x.shape)
print (y.shape)

(55000, 784)
(55000, 10)


In [7]:
X = tf.placeholder("float", [None,784])
Y_=tf.placeholder("float",[None,10])

In [8]:
def batchnorm(Ylogits,beta,scale):
    #exp_moving_avg = tf.train.ExponentialMovingAverage(0.998, iteration)
    bnepsilon = 1e-5
    mean, variance = tf.nn.moments(Ylogits, [0])
    BN = tf.nn.batch_normalization(Ylogits,mean,variance,beta,scale,bnepsilon)
    return BN

In [9]:
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30

In [10]:
S1 = tf.Variable(tf.ones([L]))
O1 = tf.Variable(tf.zeros([L]))

S2 = tf.Variable(tf.ones([M]))
O2 = tf.Variable(tf.zeros([M]))

S3 = tf.Variable(tf.ones([N]))
O3 = tf.Variable(tf.zeros([N]))

S4 = tf.Variable(tf.ones([O]))
O4 = tf.Variable(tf.zeros([O]))


In [23]:
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
#B1 = tf.Variable(tf.zeros([L]))

W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
#B2 = tf.Variable(tf.zeros([M]))

W3 = tf.Variable(tf.truncated_normal([M,N], stddev=0.1))
#B3 = tf.Variable(tf.zeros([N]))

W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.zeros([O]))

W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

In [24]:
Y1 = (tf.matmul(X, W1))
BN1=batchnorm(Y1,O1,S1)
l1_BN = tf.nn.sigmoid(BN1)

Y2 = (tf.matmul(l1_BN, W2))
BN2=batchnorm(Y2,O2,S2)
l2_BN = tf.nn.sigmoid(BN2)

Y3 = (tf.matmul(l2_BN, W3))
BN3=batchnorm(Y3,O3,S3)
l3_BN = tf.nn.sigmoid(BN3)

Y4 = (tf.matmul(l3_BN, W4))
BN4=batchnorm(Y4,O4,S4)
l4_BN = tf.nn.sigmoid(BN4)

Ylogits_BN = tf.matmul(l4_BN, W5) + B5
Ylogits_BN = tf.nn.softmax(Ylogits_BN)


In [25]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits_BN, labels=Y_)
cost = tf.reduce_mean(cost)

In [26]:
correct_prediction = tf.equal(tf.argmax(Ylogits_BN, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [27]:
# variable learning rate
lr = tf.placeholder(tf.float32)

In [28]:
# training step, the learning rate is a placeholder
optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

In [29]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

In [30]:
training_epochs = 25
batch_size = 100

In [31]:
init =tf.global_variables_initializer()

In [32]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        c=0.0
        avg_cost=0.0
        # Loop over all batches
        for i in range(total_batch):
            
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
            
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            #print(batch_X.shape)
            _,c=sess.run([optimizer,cost], feed_dict={X:batch_X,Y_:batch_Y,lr: learning_rate})
            avg_cost += c / total_batch

        print("epchos {} cross entropy={}".format(epoch+1,avg_cost))
            
    print ("Optimization Finished!")

    # Test model
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print( "Accuracy:", accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

epchos 1 cross entropy=1.8001413512229925
epchos 2 cross entropy=1.5547489666938787
epchos 3 cross entropy=1.512746886990285
epchos 4 cross entropy=1.5026151917197483
epchos 5 cross entropy=1.4956990708004336
epchos 6 cross entropy=1.4928617685491399
epchos 7 cross entropy=1.489466449780896
epchos 8 cross entropy=1.4858593355525629
epchos 9 cross entropy=1.484520356004888
epchos 10 cross entropy=1.4823024064844295
epchos 11 cross entropy=1.4802128234776564
epchos 12 cross entropy=1.4798860296336083
epchos 13 cross entropy=1.4797930830175223
epchos 14 cross entropy=1.4774635770104143
epchos 15 cross entropy=1.477598488981074
epchos 16 cross entropy=1.4764033289389193
epchos 17 cross entropy=1.4753799590197476
epchos 18 cross entropy=1.4757269072532648
epchos 19 cross entropy=1.4740860373323612
epchos 20 cross entropy=1.4743659455125981
epchos 21 cross entropy=1.4745512546192523
epchos 22 cross entropy=1.4728264125910673
epchos 23 cross entropy=1.4726136827468876
epchos 24 cross entropy=